In [ ]:
# === INSTRUCTIONS for SUBMISSIONS ===

Follow the instructions for submissions carefully.

Prepare a single Jupyter notebook, please include questions, and question numbers in the questions and in the responses.
Submit your homework notebook via the following form:

   + http://goo.gl/forms/er3OFr5eCMWDngB72
   
# === Other BACKGROUND INFORMATION ===

This Homework, you will need to use a map-reduce framework to solve all  problems involving data. You can use MrJob or raw Hadoop mapreduce streaming.

The following notebook serves as a useful Notebook to extend for this Homework (loaded with examples and boiler plate code)

   +  https://www.dropbox.com/s/pjd6maluq4ogt7m/HW02-Supporting-Material.ipynb?dl=0

Please refer to the following slides and reference material for more background


Pattern in MapReduce : 

    + Lin, Jimmy, & Dyer, Chris. (2010). Data-intensive text processing with MapReduce. 
      San Rafael, CA: Morgan & Claypool Publishers. 
      Chapter 3
      Book available here: https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf

   + Chapter 2 in this book: Hadoop with MRJob (calculater salare average etc.) 
          -  https://www.dropbox.com/s/jd3z2s216p9kc1z/hadoop-with-python-MRJOB.pdf?dl=0
          -  Source code: https://www.dropbox.com/sh/j8oettuxbgztk0p/AAAwq9PpEeByecDmaSNslnBPa?dl=0

   + http://mrjob.readthedocs.io/en/latest/

Counters in MRJob
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/5thl14n4pqvhzt5/Counter.ipynb 

KMeans Clustering (a form of Flat clustering)
   + Slides:  https://www.dropbox.com/s/6ef1mlrqr3xpnw0/Lecture07.1-Clustering.pdf?dl=0
   + http://nlp.stanford.edu/IR-book/pdf/16flat.pdf
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/dcw8evd9v0su3xu/K_Means_Unit_Test_Notebook.ipynb



# === SERVER with MRJob installed ===

http://ec2-52-201-222-181.compute-1.amazonaws.com:8000

 you will need a password to access.

Dont forget to save your notebooks REGULARLY to your local laptops (as the cluster could crash without warning)

# HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

## Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

Use the following command to grab the file:

    curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created.  

**Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

### I was unable to get our cluster on AWS working correctly. The UI Tracker for our AWS cluster (http://ec2-52-201-222-181.compute-1.amazonaws.com:8088/cluster) doesn't show anything after June 23rd when it looks like you last used it. 

In [1]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 48.5M  100 48.5M    0     0  3313k      0  0:00:15  0:00:15 --:--:-- 3849k


In [2]:
%%writefile complaint_type_counter.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob


class ComplaintTypeCount(MRJob):
    
    
    def mapper(self, _, line):
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')

        complaint_id, complaint_type, _ = line.split(',', 2)
        complaint_type = complaint_type.lower().strip().replace(' ', '_')
        del _

        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the header in the csv)
        try:
            int(complaint_id)
        except: 
            return

        if complaint_type not in ['debt_collection', 'mortgage']:
            complaint_type = 'other'
        self.increment_counter('complaint_type', complaint_type, 1)
        yield (complaint_type, 1)
        
    def combiner(self, complaint, counts):
        yield (complaint, sum(counts))
        
    def reducer(self, complaint, counts):
        yield (complaint, sum(counts))


if __name__ == '__main__':
    ComplaintTypeCount.run()

Overwriting complaint_type_counter.py


In [3]:
%reload_ext autoreload
%autoreload 2

from complaint_type_counter import ComplaintTypeCount
mr_job = ComplaintTypeCount(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

[{'complaint_type': {'other': 142788, 'debt_collection': 44372, 'mortgage': 125752}}]


# HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

## HW2.2 Part 1: 

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux

Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. 

### What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.


In [4]:
# Write the words to a file
!echo foo foo quux labs foo bar quux > WordCount.txt

In [5]:
%%writefile word_counter.py
# Create word counter
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class WordCounter(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    WordCount.run()

Overwriting word_counter.py


In [6]:
from word_counter import WordCounter

mr_job = WordCounter(args=['WordCount.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    print runner.counters()

[{'group': {'Num_mapper_calls': 1, 'Num_reducer_calls': 4}}]


### Explain what the above does here

Only 1 mapper call because the mapper is called for each line that it processes. It is only processing one line here, so 1 mapper call. The number of reducer calls corresponds to the number of keys (words). There are 4 unique words in the data, so 4 reducer calls. 

## HW2.2 Part 2:

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called.  

In [7]:
%%writefile issue_word_counter.py
# Create word counter
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
import re, string
 
class IssueWordCount(MRJob):
    
    def mapper(self, _, line):
        
        self.increment_counter('group', 'Num_mapper_calls', 1)
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()

        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)
        
    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield (word, sum(counts))

if __name__ == '__main__':
    IssueWordCount.run()

Overwriting issue_word_counter.py


In [8]:
%reload_ext autoreload
%autoreload 2

from issue_word_counter import IssueWordCount
mr_job = IssueWordCount(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output
    for line in runner.stream_output():
        print(line.strip())
    print runner.counters()

"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can"	1999
"cancelling"	2795
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company"	4858
"cont"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	55251
"credited"	92
"customer"	2734
"d"	11848
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"decrease"	1149
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"did"	139
"didn"	925
"disclosure"	5214
"disclosures"	64
"dispute"	904
"disputes"	6938
"embezzlement"	3276
"expect"	807
"false"	2508
"fee"	3198
"fees"	807
"for"	929
"forbearance"	350
"f

# HW2.2 Part 3:

### What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job?

The Mapper Counter confirms that I am in fact processing all lines of data (312913). Note that this value includes the header row in the CSV, even though I do exclude it from the word count. The reducer calls correspond to the number of keys, indicating that there are 172 unique keys (words) in this dataset. 

## HW2.2 Part 4:

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called.   

In [9]:
%%writefile issue_word_counter_combiner.py
# Create word counter
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
 
class IssueWordCountCombiner(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,  
            ),
            MRStep(
                combiner=self.combiner
            ),
            MRStep(
                reducer=self.reducer
            )
        ]
    
    def mapper(self, _, line):
        
        self.increment_counter('group', 'Num_mapper_calls', 1)
        
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()

        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)
            
    def combiner(self, word, counts):
        self.increment_counter('group', 'Num_combiner_calls', 1)
        yield (word, sum(counts))
        
    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield (word, sum(counts))

if __name__ == '__main__':
    IssueWordCountCombiner.run()

Overwriting issue_word_counter_combiner.py


In [10]:
%reload_ext autoreload
%autoreload 2

from issue_word_counter_combiner import IssueWordCountCombiner
mr_job = IssueWordCountCombiner(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output
    for line in runner.stream_output():
        print(line.strip())
    print runner.counters()

"a"	3503
"account"	20681
"acct"	163
"action"	2505
"advance"	240
"advertising"	1193
"amount"	98
"amt"	71
"an"	2505
"and"	16448
"application"	8868
"applied"	139
"apply"	118
"apr"	3431
"arbitration"	168
"are"	3821
"atm"	2422
"attempts"	11848
"available"	274
"balance"	597
"bank"	202
"bankruptcy"	222
"being"	5663
"billing"	8158
"by"	5663
"can"	1999
"cancelling"	2795
"card"	4405
"cash"	240
"caused"	5663
"changes"	350
"charged"	976
"charges"	131
"checks"	75
"closing"	2795
"club"	12545
"collect"	11848
"collection"	1907
"communication"	6920
"company"	4858
"cont"	11848
"contact"	3053
"convenience"	75
"costs"	4350
"credit"	55251
"credited"	92
"customer"	2734
"d"	11848
"day"	71
"dealing"	1944
"debit"	2422
"debt"	19309
"decision"	2774
"decrease"	1149
"delay"	243
"delinquent"	1061
"deposits"	10555
"determination"	1490
"did"	139
"didn"	925
"disclosure"	5214
"disclosures"	64
"dispute"	904
"disputes"	6938
"embezzlement"	3276
"expect"	807
"false"	2508
"fee"	3198
"fees"	807
"for"	929
"forbearance"	350
"f

### What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job?

Num_mapper_calls = 312913 (number of rows in CSV). Num_combiner_calls = 483 (number of times the combiner ran across all nodes - note that the combiner runs on all unique keys in each node). Num_reducer_calls = 172 (number of unique keys/words)

## HW2.2.1:

Using a single reducer perform a sort of the words in decreasing order of word counts. 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.


In [11]:
%%writefile issue_word_counter_sort.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class IssueWordCountSort(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):        
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()

        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)
            
    def combiner(self, word, counts):
        yield (word, sum(counts))
        
    def reducer(self, word, counts):
        count = sum(counts)
        yield (word, count)
        
    def mapper_sort(self, word, count):
        # take a (very) large number and minus the count to 
        # reverse the way MapReduce normally sorts (smallest to largest)
        value = 1e6 - count
        yield ((value, word), None)
        
    def reducer_sort(self, value_word, _):
        # this is taking care of ties. Map reduce will order on word second if there is a tie. 
        value, word = value_word
        yield (word, int(abs(value - 1e6)))

if __name__ == '__main__':
    IssueWordCountSort.run()

Overwriting issue_word_counter_sort.py


In [12]:
%reload_ext autoreload
%autoreload 2

from issue_word_counter_sort import IssueWordCountSort
mr_job = IssueWordCountSort(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output
    for line in runner.stream_output():
        print(line.strip())

"loan"	119630
"modification"	70487
"credit"	55251
"servicing"	36767
"report"	34903
"incorrect"	29133
"information"	29069
"on"	29069
"or"	22533
"account"	20681
"debt"	19309
"and"	16448
"opening"	16205
"club"	12545
"health"	12545
"not"	12353
"attempts"	11848
"collect"	11848
"cont"	11848
"d"	11848
"owed"	11848
"of"	10885
"my"	10731
"deposits"	10555
"withdrawals"	10555
"problems"	9484
"application"	8868
"to"	8401
"unable"	8178
"billing"	8158
"other"	7886
"disputes"	6938
"communication"	6920
"tactics"	6920
"reporting"	6559
"lease"	6337
"the"	6248
"being"	5663
"by"	5663
"caused"	5663
"funds"	5663
"low"	5663
"process"	5505
"disclosure"	5214
"verification"	5214
"managing"	5006
"company"	4858
"investigation"	4858
"s"	4858
"identity"	4729
"card"	4405
"get"	4357
"score"	4357
"costs"	4350
"settlement"	4350
"improper"	4309
"interest"	4238
"protection"	4139
"when"	4095
"repaying"	3844
"your"	3844
"fraud"	3842
"are"	3821
"pay"	3821
"you"	3821
"taking"	3747
"a"	3503
"apr"	3431
"rate"	3431
"embezzlemen

### Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. 

In [13]:
%reload_ext autoreload
%autoreload 2

# Note that I dealt with the potential tie issue in my MRJob above.
from issue_word_counter_sort import IssueWordCountSort
mr_job = IssueWordCountSort(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 49:
            break

"loan"	119630
"modification"	70487
"credit"	55251
"servicing"	36767
"report"	34903
"incorrect"	29133
"information"	29069
"on"	29069
"or"	22533
"account"	20681
"debt"	19309
"and"	16448
"opening"	16205
"club"	12545
"health"	12545
"not"	12353
"attempts"	11848
"collect"	11848
"cont"	11848
"d"	11848
"owed"	11848
"of"	10885
"my"	10731
"deposits"	10555
"withdrawals"	10555
"problems"	9484
"application"	8868
"to"	8401
"unable"	8178
"billing"	8158
"other"	7886
"disputes"	6938
"communication"	6920
"tactics"	6920
"reporting"	6559
"lease"	6337
"the"	6248
"being"	5663
"by"	5663
"caused"	5663
"funds"	5663
"low"	5663
"process"	5505
"disclosure"	5214
"verification"	5214
"managing"	5006
"company"	4858
"investigation"	4858
"s"	4858
"identity"	4729


### Present bottom 10 tokens (least frequent items). 

In [14]:
%%writefile issue_word_counter_lowest_first.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class IssueWordCountLowestFirst(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):        
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()

        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)
            
    def combiner(self, word, counts):        
        yield (word, sum(counts))
        
    def reducer(self, word, counts):
        count = sum(counts)
        yield (word, count)
        
    def mapper_sort(self, word, count):
        # adding leading zeros to deal with sort issues
        count = str(count).zfill(6)
        yield ((count, word), None)
        
    def reducer_sort(self, count_word, _):
        count, word = count_word
        yield (word, int(count))

if __name__ == '__main__':
    IssueWordCountLowestFirst.run()

Overwriting issue_word_counter_lowest_first.py


In [15]:
%reload_ext autoreload
%autoreload 2

from issue_word_counter_lowest_first import IssueWordCountLowestFirst
mr_job = IssueWordCountLowestFirst(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 9:
            break

"disclosures"	64
"missing"	64
"amt"	71
"day"	71
"checks"	75
"convenience"	75
"credited"	92
"payment"	92
"amount"	98
"apply"	118


# HW2.2.2:

**Using 3 reducers: 

Repeat HW2.2.1 using 3 reducers. Use the same code as in HW2.2.1  with just one modification to the command line: just add --jobconf mapred.reduce.tasks=3 as see presented here: 

    python HW2.2WordCount.py --jobconf mapred.reduce.tasks=3 oneLinerTextFile.txt

Describe what you see. Is this correct?

In [16]:
# I'm using mrjob runner instead of command line since that's what I've been doing above

# Full sort - most frequent words first

%reload_ext autoreload
%autoreload 2

from issue_word_counter_sort import IssueWordCountSort
mr_job = IssueWordCountSort(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output
    for line in runner.stream_output():
        print(line.strip())

"loan"	119630
"modification"	70487
"credit"	55251
"servicing"	36767
"report"	34903
"incorrect"	29133
"information"	29069
"on"	29069
"or"	22533
"account"	20681
"debt"	19309
"and"	16448
"opening"	16205
"club"	12545
"health"	12545
"not"	12353
"attempts"	11848
"collect"	11848
"cont"	11848
"d"	11848
"owed"	11848
"of"	10885
"my"	10731
"deposits"	10555
"withdrawals"	10555
"problems"	9484
"application"	8868
"to"	8401
"unable"	8178
"billing"	8158
"other"	7886
"disputes"	6938
"communication"	6920
"tactics"	6920
"reporting"	6559
"lease"	6337
"the"	6248
"being"	5663
"by"	5663
"caused"	5663
"funds"	5663
"low"	5663
"process"	5505
"disclosure"	5214
"verification"	5214
"managing"	5006
"company"	4858
"investigation"	4858
"s"	4858
"identity"	4729
"card"	4405
"get"	4357
"score"	4357
"costs"	4350
"settlement"	4350
"improper"	4309
"interest"	4238
"protection"	4139
"when"	4095
"repaying"	3844
"your"	3844
"fraud"	3842
"are"	3821
"pay"	3821
"you"	3821
"taking"	3747
"a"	3503
"apr"	3431
"rate"	3431
"embezzlemen

In [17]:
# Top 50 items

%reload_ext autoreload
%autoreload 2

# Note that I dealt with the potential tie issue in my MRJob above.
from issue_word_counter_sort import IssueWordCountSort
mr_job = IssueWordCountSort(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 49:
            break

"loan"	119630
"modification"	70487
"credit"	55251
"servicing"	36767
"report"	34903
"incorrect"	29133
"information"	29069
"on"	29069
"or"	22533
"account"	20681
"debt"	19309
"and"	16448
"opening"	16205
"club"	12545
"health"	12545
"not"	12353
"attempts"	11848
"collect"	11848
"cont"	11848
"d"	11848
"owed"	11848
"of"	10885
"my"	10731
"deposits"	10555
"withdrawals"	10555
"problems"	9484
"application"	8868
"to"	8401
"unable"	8178
"billing"	8158
"other"	7886
"disputes"	6938
"communication"	6920
"tactics"	6920
"reporting"	6559
"lease"	6337
"the"	6248
"being"	5663
"by"	5663
"caused"	5663
"funds"	5663
"low"	5663
"process"	5505
"disclosure"	5214
"verification"	5214
"managing"	5006
"company"	4858
"investigation"	4858
"s"	4858
"identity"	4729


In [18]:
# 10 least frequent items: 

%reload_ext autoreload
%autoreload 2

from issue_word_counter_lowest_first import IssueWordCountLowestFirst
mr_job = IssueWordCountLowestFirst(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=3"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 9:
            break

"disclosures"	64
"missing"	64
"amt"	71
"day"	71
"checks"	75
"convenience"	75
"credited"	92
"payment"	92
"amount"	98
"apply"	118


## Yes, it is still correct. I wrote my code to start with so that it would work no matter how many reducers we have. HW said to run it using a single reducer, not that I had to write it such that it would only work on one reducer, so hopefully that isn't an issue.

# HW 2.2.3: [Optional; we will cover this in class]

Solve the "total sort" issue in HW2.2.2.

## First version of my code works. In order for this sort to work, you need to have a single key in the mapreduce sort steps, which forces the sort to occur on one reducer (even if you allow multiple reducers in earlier steps).

# HW2.3: Shopping Cart Analysis:

Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 

In [19]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 -o ProductPurchaseData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3377k  100 3377k    0     0   759k      0  0:00:04  0:00:04 --:--:-- 1225k


## HW2.3 Part 1:

Do some exploratory data analysis of this dataset guided by the following questions:

How many unique items are available from this supplier?

In [20]:
items = []

input_file = open('ProductPurchaseData.txt',"r+")

for row in input_file:
    line = row.split()
    for item in line:
        if item in items:
            continue
        else:
            items.append(item)
            
print('Number of unique items', len(items))
    

('Number of unique items', 12592)


## HW2.3 Part 2:

### Using a single reducer: 

Report your findings using Hadoop Map-Reduce:


+ number of unique products 

In [21]:
%%writefile shopping_unique.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string


class ShoppingUnique(MRJob):
    
    def mapper(self, _, line):
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        basket = basket.split()
        
        for item in basket:
            yield (item, 1)
            
    def combiner(self, item, counts):
        yield (item, sum(counts))
        
    def reducer(self, item, counts):
        # This counter will give me the number of unique words
        # Reducer runs once for each key, each unique word = 1 key
        self.increment_counter('group', 'Num_reducer_calls/Unique Items', 1)
        count = sum(counts)
        yield (item, count)

if __name__ == '__main__':
    ShoppingUnique.run()

Overwriting shopping_unique.py


In [22]:
%reload_ext autoreload
%autoreload 2

from shopping_unique import ShoppingUnique

mr_job = ShoppingUnique(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

[{'group': {'Num_reducer_calls/Unique Items': 12592}}]


+ largest basket

In [23]:
%%writefile shopping_largest_basket.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string, uuid

class ShoppingLargestBasket(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        basket = basket.split()
        basket_size = len(basket)
        
        # yielding a unique id to clean up output (don't need to output everything in the basket)
        yield (str(uuid.uuid4()), basket_size)
        
    def mapper_sort(self, basket, basket_size):
        # take a (very) large number and minus the count to 
        # reverse the way MapReduce normally sorts (smallest to largest)
        value = 1e6 - basket_size
        yield ((value, basket), None)
        
    def reducer_sort(self, value_basket, _):
        # this is taking care of ties. Map reduce will order on word second if there is a tie. 
        value, basket = value_basket
        yield ("Basket Size", int(abs(value - 1e6)))

if __name__ == '__main__':
    ShoppingLargestBasket.run()

Overwriting shopping_largest_basket.py


In [24]:
%reload_ext autoreload
%autoreload 2

from shopping_largest_basket import ShoppingLargestBasket
mr_job = ShoppingLargestBasket(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        # Stop after providing the biggest basket
        if count > 0:
            break

"Basket Size"	37


+ report the top 50 most frequently purchased items and their frequency (break ties by sorting the products alphabetical order)

In [25]:
%%writefile shopping_top_50.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

class ShoppingTop50(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        basket = basket.split()
        
        for item in basket:
            yield (item, 1)
            
    def combiner(self, item, counts):
        yield (item, sum(counts))
        
    def reducer(self, item, counts):
        count = sum(counts)
        yield (item, count)
        
    def mapper_sort(self, word, count):
        # take a (very) large number and minus the count to 
        # reverse the way MapReduce normally sorts (smallest to largest)
        value = 1e6 - count
        yield ((value, word), None)
        
    def reducer_sort(self, value_word, _):
        # this is taking care of ties. Map reduce will order on word second if there is a tie. 
        value, word = value_word
        yield (word, int(abs(value - 1e6)))

if __name__ == '__main__':
    ShoppingTop50.run()

Overwriting shopping_top_50.py


In [26]:
%reload_ext autoreload
%autoreload 2

from shopping_top_50 import ShoppingTop50
mr_job = ShoppingTop50(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 49:
            break

"dai62779"	6667
"fro40251"	3881
"ele17451"	3875
"gro73461"	3602
"sna80324"	3044
"ele32164"	2851
"dai75645"	2736
"sna45677"	2455
"fro31317"	2330
"dai85309"	2293
"ele26917"	2292
"fro80039"	2233
"gro21487"	2115
"sna99873"	2083
"gro59710"	2004
"gro71621"	1920
"fro85978"	1918
"gro30386"	1840
"ele74009"	1816
"gro56726"	1784
"dai63921"	1773
"gro46854"	1756
"ele66600"	1713
"dai83733"	1712
"fro32293"	1702
"ele66810"	1697
"sna55762"	1646
"dai22177"	1627
"fro78087"	1531
"ele99737"	1516
"ele34057"	1489
"gro94758"	1489
"fro35904"	1436
"fro53271"	1420
"sna93860"	1407
"sna90094"	1390
"gro38814"	1352
"ele56788"	1345
"gro61133"	1321
"dai88807"	1316
"ele74482"	1316
"ele59935"	1311
"sna96271"	1295
"dai43223"	1290
"ele91337"	1289
"gro15017"	1275
"dai31081"	1261
"gro81087"	1220
"dai22896"	1219
"gro85051"	1214


# HW2.3.1 OPTIONAL

### Using 2 reducers:  

Report your findings using Hadoop Map-Reduce: 

+ number of unique products 
+ largest basket
+ report the top 50 most frequently purchased items, and their frequency (break ties by sorting the products alphabetical order)

In [27]:
# Unique Items:

%reload_ext autoreload
%autoreload 2

from shopping_unique import ShoppingUnique

mr_job = ShoppingUnique(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    print runner.counters()

[{'group': {'Num_reducer_calls/Unique Items': 12592}}]


In [28]:
# Largest basket size:

%reload_ext autoreload
%autoreload 2

from shopping_largest_basket import ShoppingLargestBasket
mr_job = ShoppingLargestBasket(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        # Stop after providing the biggest basket
        if count > 0:
            break

"Basket Size"	37


In [29]:
# Top 50 items purchased:

%reload_ext autoreload
%autoreload 2

from shopping_top_50 import ShoppingTop50
mr_job = ShoppingTop50(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=2"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        print(line.strip())
        count += 1
        if count > 49:
            break

"dai62779"	6667
"fro40251"	3881
"ele17451"	3875
"gro73461"	3602
"sna80324"	3044
"ele32164"	2851
"dai75645"	2736
"sna45677"	2455
"fro31317"	2330
"dai85309"	2293
"ele26917"	2292
"fro80039"	2233
"gro21487"	2115
"sna99873"	2083
"gro59710"	2004
"gro71621"	1920
"fro85978"	1918
"gro30386"	1840
"ele74009"	1816
"gro56726"	1784
"dai63921"	1773
"gro46854"	1756
"ele66600"	1713
"dai83733"	1712
"fro32293"	1702
"ele66810"	1697
"sna55762"	1646
"dai22177"	1627
"fro78087"	1531
"ele99737"	1516
"ele34057"	1489
"gro94758"	1489
"fro35904"	1436
"fro53271"	1420
"sna93860"	1407
"sna90094"	1390
"gro38814"	1352
"ele56788"	1345
"gro61133"	1321
"dai88807"	1316
"ele74482"	1316
"ele59935"	1311
"sna96271"	1295
"dai43223"	1290
"ele91337"	1289
"gro15017"	1275
"dai31081"	1261
"gro81087"	1220
"dai22896"	1219
"gro85051"	1214


# Didn't assume 1 reducer in first version. Wrote code so that it would work with any number of reducers

# HW2.4 (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

       SUPPORT
       In data mining, the support value of X (where X is a collection of cooccurring items sometimes referred to as
       an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each 
       row is a transaction such as a basket of items that have been purchased)  is defined as the proportion 
       of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

       CONFIDENCE 
       The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally
       a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of
       transactions T, is the proportion of the transactions that contains X which also contains Y. (Think of it as
       tgePr(Y|X) )

       The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the
       confidence and support. 
       
       Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes
       when X occurs by itself in transactions.

## HW2.4 Part 1:

Suppose we want to recommend new products to the customer based on the products they have already browsed on the online website. 

+ Write a map-reduce program to find products which are frequently browsed together. 
    + Fix the support count (cooccurence count) to s = 100 (i.e. product pairs need to occur together at least 100 times to be considered frequent) 

## Note that the way I read this, you wanted us to get the counts for the full data set, then do the top 50. Top 50 is below. 

In [30]:
%%writefile shopping_pairs.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string


class ShoppingPairs(MRJob):
    
    def mapper(self, _, line):
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        # Set function removes any duplicates
        basket = list(set(basket.split()))
        # Sorts everything in the baseket alphabetically so that we know
        # that all the item pairs will be in alphabetical order
        basket.sort()
        
        for index, item_1 in enumerate(basket):
            for item_2 in basket[index + 1:]:
                yield ((item_1, item_2), 1)
            
    def combiner(self, item_pair, counts):
        yield (item_pair, sum(counts))
        
    def reducer(self, item_pair, counts):
        count = sum(counts)
        # Only yield pairs that occurr at least 100 times
        if count > 99:
            yield (item_pair, count)

if __name__ == '__main__':
    ShoppingPairs.run()

Writing shopping_pairs.py


In [31]:
%reload_ext autoreload
%autoreload 2

from shopping_pairs import ShoppingPairs
mr_job = ShoppingPairs(args=['ProductPurchaseData.txt']) 
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        print(line.strip())

["dai16732", "fro78087"]	106
["dai18527", "sna44451"]	102
["dai22177", "dai31081"]	127
["dai22177", "dai62779"]	382
["dai22177", "dai63921"]	136
["dai22177", "dai75645"]	123
["dai22177", "dai83733"]	126
["dai22177", "dai85309"]	172
["dai22177", "ele17451"]	203
["dai22177", "ele26917"]	134
["dai22177", "ele32164"]	155
["dai22177", "ele34057"]	107
["dai22177", "ele56788"]	134
["dai22177", "ele66600"]	101
["dai22177", "ele66810"]	105
["dai22177", "ele74009"]	108
["dai22177", "ele91337"]	150
["dai22177", "fro31317"]	160
["dai22177", "fro32293"]	128
["dai22177", "fro40251"]	181
["dai22177", "fro66272"]	130
["dai22177", "fro78087"]	107
["dai22177", "fro80039"]	152
["dai22177", "fro85978"]	156
["dai22177", "gro21487"]	122
["dai22177", "gro30386"]	106
["dai22177", "gro46854"]	160
["dai22177", "gro59710"]	120
["dai22177", "gro71621"]	132
["dai22177", "gro73461"]	248
["dai22177", "sna45677"]	145
["dai22177", "sna55762"]	102
["dai22177", "sna80324"]	140
["dai22177", "sna99873"]	148
["dai22240", "

## HW2.4 Part 2:

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support for frequent (100>count) itemsets of size 2. 

+ Fix the ordering of the pairs lexicographically (left to right)
+ break ties in support (between pairs, if any exist) by taking the first ones in lexicographically increasing order
+ Report  the compute time for the Pairs job. 
+ Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
+ Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [32]:
%%writefile shopping_pairs_top_50.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string

stored_data = {}


class ShoppingPairsTop50(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        # Set function removes any duplicates
        basket = list(set(basket.split()))
        # Sorts everything in the baseket alphabetically so that we know
        # that all the item pairs will be in alphabetical order
        basket.sort()
        
        for index, item_1 in enumerate(basket):
            for item_2 in basket[index + 1:]:
                yield ((item_1, item_2), 1)
        yield (("", "total_count"), 1)

    def combiner(self, item_pair, counts):
        self.increment_counter('group', 'Num_combiner_calls', 1)
        yield (item_pair, sum(counts))
        
    def reducer(self, item_pair, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        count = sum(counts)
        if len(item_pair[0]) < 1:
            stored_data[item_pair[1]] = count
            return
        if count > 99:
            yield (item_pair, count)

    def mapper_sort(self, item_pair, count):
        self.increment_counter('group', 'Num_mapper_sort_calls', 1)
        # take a (very) large number and minus the count to 
        # reverse the way MapReduce normally sorts (smallest to largest)
        value = 1e6 - count
        yield ((value, item_pair), None)        
        
    def reducer_sort(self, value_item_pair, _):
        self.increment_counter('group', 'Num_reducer_sort_calls', 1)
        # this is taking care of ties. Map reduce will order on word second if there is a tie. 
        value, item_pair = value_item_pair
        freq = int(abs(value - 1e6))
        relative_freq = (int(abs(value - 1e6)) / stored_data["total_count"])
        freqs = (freq, relative_freq)
        yield (item_pair, freqs)

if __name__ == '__main__':
    ShoppingPairsTop50.run()

Writing shopping_pairs_top_50.py


In [33]:
%%writefile shopping_pairs_top_50_runner.py

# Top 50 items pairs by frequency:

from shopping_pairs_top_50 import ShoppingPairsTop50
mr_job = ShoppingPairsTop50(args=['ProductPurchaseData.txt', '--jobconf="mapred.map.tasks=4"', '--jobconf="mapred.reduce.tasks=4"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        print '{}, {}, {}, {}'.format(key[0], key[1], value[0], value[1])
        count += 1
        if count > 49:
            break
    print runner.counters()

Writing shopping_pairs_top_50_runner.py


In [34]:
%time !python2.7 shopping_pairs_top_50_runner.py

dai62779, ele17451, 1592, 0.0511880646925
fro40251, sna80324, 1412, 0.0454004694383
dai75645, fro40251, 1254, 0.0403202469374
fro40251, gro85051, 1213, 0.0390019613517
dai62779, gro73461, 1139, 0.0366226166361
dai75645, sna80324, 1130, 0.0363332368734
dai62779, fro40251, 1070, 0.0344040384554
dai62779, sna80324, 923, 0.0296775023311
dai62779, dai85309, 918, 0.0295167357963
ele32164, gro59710, 911, 0.0292916626475
dai62779, dai75645, 882, 0.0283592167454
fro40251, gro73461, 882, 0.0283592167454
dai62779, ele92920, 877, 0.0281984502106
fro40251, fro92469, 835, 0.026848011318
dai62779, ele32164, 832, 0.0267515513971
dai75645, gro73461, 712, 0.0228931545609
dai43223, ele32164, 711, 0.022861001254
dai62779, gro30386, 709, 0.02279669464
ele17451, fro40251, 697, 0.0224108549564
dai85309, ele99737, 659, 0.0211890292917
dai62779, ele26917, 650, 0.020899649529
gro21487, gro73461, 631, 0.0202887366966
dai62779, sna45677, 604, 0.0194205974084
ele17451, sna80324, 597, 0.0191955242597
dai62779, gro7

### Above is done on a single computer, OS X, quad core,  4 map and 4 reduce tasks (set in runner). Output includes all other requested information. 

# HW2.5: Stripes

Repeat 2.4 using the stripes design pattern for finding cooccuring pairs (and out.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs

In [35]:
%%writefile shopping_stripes_top_50.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
from collections import Counter

stored_data = {}


class ShoppingStripesTop50(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            ),
            MRStep(
                mapper=self.mapper_sort,
                reducer=self.reducer_sort
            )
        ]
    
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        basket = regex.sub(' ', line.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        basket = re.sub( '\s+', ' ', basket)
        
        # Creates a list of items in each basket
        # Set function removes any duplicates
        basket = list(set(basket.split()))
        
        # Sorts everything in the baseket alphabetically so that we know
        # that all the item pairs will be in alphabetical order
        basket.sort()
        
        for index, item_1 in enumerate(basket):
            counts = Counter()
            for item_2 in basket[index + 1:]:
                counts[item_2] += 1
                
            # yielding a dictionary with the first item as the key
            # and the value as a dictionary with the other items that
            # occured with it and how many times they occured
            yield(item_1, counts)
            
        yield ("TOTAL_COUNT", {'TOTAL_COUNT': 1})

    def combiner(self, item, counters):
        self.increment_counter('group', 'Num_combiner_calls', 1)
        counts = Counter()
        for c in counters: 
            for item_2, count in c.items():
                counts[item_2] += count
                
        yield (item, counts)
        
    def reducer(self, item, counters):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        counts = Counter()
        for c in counters: 
            for item_2, count in c.items():
                counts[item_2] += count
        
        if item == "TOTAL_COUNT":
            stored_data[item] = counts[item]
            return
        
        for item_2, count in counts.items():
            if count > 99:
                yield ((item, item_2), count)
        
    def mapper_sort(self, item_pair, count):
        self.increment_counter('group', 'Num_mapper_sort_calls', 1)
        # take a (very) large number and minus the count to 
        # reverse the way MapReduce normally sorts (smallest to largest)
        value = 1e6 - count
        yield ((value, item_pair), None)        
        
    def reducer_sort(self, value_item_pair, _):
        self.increment_counter('group', 'Num_reducer_sort_calls', 1)
        # this is taking care of ties. Map reduce will order on word second if there is a tie. 
        value, item_pair = value_item_pair
        freq = int(abs(value - 1e6))
        relative_freq = (int(abs(value - 1e6)) / stored_data["TOTAL_COUNT"])
        freqs = (freq, relative_freq)
        yield (item_pair, freqs)

if __name__ == '__main__':
    ShoppingStripesTop50.run()

Writing shopping_stripes_top_50.py


In [36]:
%%writefile shopping_stripes_top_50_runner.py

# Top 50 items pairs by frequency:

from shopping_stripes_top_50 import ShoppingStripesTop50
mr_job = ShoppingStripesTop50(args=['ProductPurchaseData.txt', '--jobconf="mapred.map.tasks=4"', '--jobconf="mapred.reduce.tasks=4"']) 
with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        print '{}, {}, {}, {}'.format(key[0], key[1], value[0], value[1])
        count += 1
        if count > 49:
            break
    print runner.counters()

Writing shopping_stripes_top_50_runner.py


In [37]:
%time !python2.7 shopping_stripes_top_50_runner.py

dai62779, ele17451, 1592, 0.0511880646925
fro40251, sna80324, 1412, 0.0454004694383
dai75645, fro40251, 1254, 0.0403202469374
fro40251, gro85051, 1213, 0.0390019613517
dai62779, gro73461, 1139, 0.0366226166361
dai75645, sna80324, 1130, 0.0363332368734
dai62779, fro40251, 1070, 0.0344040384554
dai62779, sna80324, 923, 0.0296775023311
dai62779, dai85309, 918, 0.0295167357963
ele32164, gro59710, 911, 0.0292916626475
dai62779, dai75645, 882, 0.0283592167454
fro40251, gro73461, 882, 0.0283592167454
dai62779, ele92920, 877, 0.0281984502106
fro40251, fro92469, 835, 0.026848011318
dai62779, ele32164, 832, 0.0267515513971
dai75645, gro73461, 712, 0.0228931545609
dai43223, ele32164, 711, 0.022861001254
dai62779, gro30386, 709, 0.02279669464
ele17451, fro40251, 697, 0.0224108549564
dai85309, ele99737, 659, 0.0211890292917
dai62779, ele26917, 650, 0.020899649529
gro21487, gro73461, 631, 0.0202887366966
dai62779, sna45677, 604, 0.0194205974084
ele17451, sna80324, 597, 0.0191955242597
dai62779, gro7